# Coffea and Awkward

**March 15, 2023 at Beijing**

shift + command + k = delete line
command + / = annotating code

In [1]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
from coffea.lumi_tools import LumiMask

/tmp/ipykernel_310944/3220351898.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


### coffea读文件

In [2]:
import input_file 
reload(input_file)
# test_events = input_file.input_file('test')# 
###############  DATASETS
##### Electron channel
# dataC_ele_events = input_file.input_file('dataC_ele')# 15min
# dataD_ele_events = input_file.input_file('dataD_ele')# 5min
# dataE_ele_events = input_file.input_file('dataE_ele')# 8min
# dataF_ele_events = input_file.input_file('dataF_ele')# 26min
# dataG_ele_events = input_file.input_file('dataG_ele')# 4min

##### Muon channel
# dataC_muon_events = input_file.input_file('dataC_muon')# 11min
# dataD_muon_events = input_file.input_file('dataD_muon')# 5min
# dataE_muon_events = input_file.input_file('dataE_muon')# 4min no tree55！！！
# dataF_muon_events = input_file.input_file('dataF_muon')# --------
# dataG_muon_events = input_file.input_file('dataG_muon')# 4min


###############  MC
##### 2022
# zg0_22_events = input_file.input_file('zg0_22')# 5min
# zg1_22_events = input_file.input_file('zg1_22')# 1min part!!!
# zg2_22_events = input_file.input_file('zg2_22')# 1min
zg3_22_events = input_file.input_file('zg3_22')# 1min
# ttbar_22_events = input_file.input_file('tt_22')# 2min
# wz_22_events = input_file.input_file('wz_22')# 2min
# ww_22_events = input_file.input_file('ww_22')# 1min
# zz_22_events = input_file.input_file('zz_22')# 1min
# dy_22_events = input_file.input_file('dy_22')# 2min

##### 2022EE
# zg0_22EE_events = input_file.input_file('zg0_22EE')# 5min
# zg1_22EE_events = input_file.input_file('zg1_22EE')# 2min part!!!
# zg2_22EE_events = input_file.input_file('zg2_22EE')# 1min
# zg3_22EE_events = input_file.input_file('zg3_22EE')# 1min
# ttbar_22EE_events = input_file.input_file('tt_22EE')# 4min
# wz_22EE_events = input_file.input_file('wz_22EE')# 2min30
# ww_22EE_events = input_file.input_file('ww_22EE')# 1min
# zz_22EE_events = input_file.input_file('zz_22EE')# 1min
# dy_22EE_events = input_file.input_file('dy_22EE')# ----------



#muon channel 19min
#ele channel 

zg3_22  is done!


In [14]:
EVENT_ELE = {
    # 'test': test_events,
    # 'dataC_ele':dataC_ele_events,
    # 'dataD_ele':dataD_ele_events,
    # # 'dataE_ele':dataE_ele_events,
    # # 'dataF':dataF_events,
    # # 'dataG':dataG_events,

    # 'zg0_22': zg0_22_events,
    # 'zg1_22': zg1_22_events,
    'zg2_22': zg2_22_events,
    'zg3_22': zg3_22_events,
    # 'tt_22': ttbar_22_events,
    # 'zz_22': zz_22_events, #No LHEpdf
    # 'wz_22': wz_22_events,
    # 'ww_22': ww_22_events,  
    'dy_22': dy_22_events,

    # 'zg2_22EE': zg2_events,
    # 'zg3_22EE': zg3_events,
    # 'tt': ttbar_events,
    # 'zz': zz_events,
    # 'wz': wz_events,
    # 'ww': ww_events,
    # 'dy_22': dy_22_events, #一个要求prompt，一个要求nonprompt

}

In [17]:
photon = EVENT_MUON['zg3_22'][0][ak.num(EVENT_MUON['zg3_22'][0].Photon.pt >50)>=1]
EVENT_MUON['zg3_22'][0].event[~photon.event]

ValueError: in NumpyArray attempting to get -34279, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/cpu-kernels/awkward_regularize_arrayslice.cpp#L14)

In [13]:
EVENT_MUON['zg3_22'][0].event

<Array [34278, 34302, ... 582619, 582610] type='4698 * uint64[parameters={"__doc...'>

In [3]:
EVENT_MUON = {
    # 'test': test_events,
    # 'dataC_muon':dataC_muon_events,
    # 'dataD_muon':dataD_muon_events,
    # 'dataE_muon':dataE_muon_events,
    # 'dataF':dataF_events,
    # 'dataG':dataG_events,
    # 'zg0_22': zg0_22_events,
    # 'zg1_22': zg1_22_events,
    # 'zg2_22': zg2_22_events,
    'zg3_22': zg3_22_events,
    # 'tt_22': ttbar_22_events,
    # 'zz_22': zz_22_events,
    # 'wz_22': wz_22_events,
    # 'ww_22': ww_22_events, 
    # 'dy_22': dy_22_events, #一个要求prompt，一个要求nonprompt

}

## selection

In [20]:
import selection_cut
reload(selection_cut)
event_ele_phodrs = {key: {} for key in EVENT_ELE}

for key in EVENT_ELE.keys():
    event_ele_variables = []
    for i in EVENT_ELE[key]:
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
        elif 'dataC' in key:
            i = selection_cut.run_num(i,'C')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataD' in key:
            i = selection_cut.run_num(i,'D')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataE' in key:
            i = selection_cut.run_num(i,'E')
            event_weight =  {'event_weight':'NONE'}
        # elif key == 'dataF':
        #     murun = selection_cut.muon(i,'run_num','F')
        #     event_weight =  {'event_weight':'NONE'}
        #     mu0 = selection_cut.muon(murun,'num',True)
        elif key == 'dataG':
            i = selection_cut.run_num(i,'G')
            event_weight =  {'event_weight':'NONE'}
        leptons,photons,event = selection_cut.electron_nTuple(i,key,"D")
        # mass_ee = selection_cut.MASS_Ele(leptons,photons,'ee')
        # mass_gee = selection_cut.MASS_Ele(leptons,photons,'gee')
        # event_ele_variables.append(selection_cut.VARIABLES('ele',key,leptons,photons,event,mass_ee,mass_gee,event_weight))    
        event_ele_variables.append(selection_cut.VARIABLES('ele',key,leptons,photons,event,False,False,event_weight))    


    event_ele_phodrs[key] = event_ele_variables
    print(key,'event_phodrs',event_ele_phodrs[key][0])




#dataC_ele 54min
#dataD_ele 4min
#zg0_22 #19min
#zg1_22 2min
#zg2_22 1min
#zg3_22 1min
#tt_22 25min
#ww,wz,zz 5min
#total 113min

dataC_ele event_phodrs {'photon_pt': <Array [] type='0 * ?float32[parameters={"__doc__": "pt"}]'>, 'photon_eta': <Array [] type='0 * ?float32[parameters={"__doc__": "eta"}]'>}
dataD_ele event_phodrs {'photon_pt': <Array [] type='0 * ?float32[parameters={"__doc__": "pt"}]'>, 'photon_eta': <Array [] type='0 * ?float32[parameters={"__doc__": "eta"}]'>}


In [21]:
# ele!!!!
# sample_names = ["dataC_ele","dataD_ele","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#,
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22"]#"tt",
sample_names = ["dataC_ele","dataD_ele"]#,"dy""zg2", "zg3"

# 初始化一个字典，用于存储合并后的数据
event_ele_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_ele_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_ele_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_ele_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_ele_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)


#all 95min

dataC_ele
dataD_ele


In [22]:
event_ele_final

{'dataC_ele': {'photon_pt': <Array [33.2, 35, 34.9, ... 35.6, 35.1, 34.6] type='36 * ?float32[parameters={"_...'>,
  'photon_eta': <Array [2.47, -0.557, -0.45, ... 2.39, 2.43] type='36 * ?float32[parameters={"__...'>},
 'dataD_ele': {'photon_pt': <Array [35, 30.9, 42.7, ... 31.6, 36.3, 31.6] type='22 * ?float32[parameters={"_...'>,
  'photon_eta': <Array [-0.00108, 2.16, ... -0.733, -2.13] type='22 * ?float32[parameters={"__do...'>}}

In [23]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'ele1_pt', 'ele2_pt', 'ele1_eta', 'ele2_eta', 'ele_mass','gee_mass','photon_endcap','photon_barrel']#,'npvsGood','Rho_Calo','Rho_tracker']#,,,'dr_lg','charge1','charge2'
    data_subkeys = ['photon_pt','photon_eta']#['photon_r9','photon_seedGain','run']#'npvsGood','Rho_Calo','Rho_tracker'#'photon_endcap','photon_barrel'
    # other_subkeys = data_subkeys + ['LHEPdfWeight','LHEScaleWeight','PSWeight']#['generator_weight', 'event_weight']#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_ele1_pt','gen_ele2_pt','gen_ele1_eta','gen_ele2_eta','gen_gmumu','gen_mumu',,'gen_gmumu'
    other_subkeys = data_subkeys 

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [24]:
# 存储数据
# save_event_dict(event_ele_final, 'selection_ele_file/v2/A/event_ele_final')
# save_event_dict(event_ele_final, 'selection_ele_file/v2/B/event_ele_final')
# save_event_dict(event_ele_final, 'selection_ele_file/v2/C/event_ele_final')
save_event_dict(event_ele_final, 'selection_ele_file/v2/D/event_ele_final')

# save_event_dict(event_ele_final, 'ABCD/ABCD_file/ele_channel/A/event_ele_final')

In [4]:
import selection_cut
reload(selection_cut)
event_muon_phodrs = {key: {} for key in EVENT_MUON}

for key in EVENT_MUON.keys():
    event_muon_variables = []
    for i in EVENT_MUON[key]:
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
        elif 'dataC' in key:
            i = selection_cut.run_num(i,'C')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataD' in key:
            i = selection_cut.run_num(i,'D')
            event_weight =  {'event_weight':'NONE'}
        elif 'dataE' in key:
            i = selection_cut.run_num(i,'E')
            event_weight =  {'event_weight':'NONE'}
        # elif key == 'dataF':
        #     murun = selection_cut.muon(i,'run_num','F')
        #     event_weight =  {'event_weight':'NONE'}
        #     mu0 = selection_cut.muon(murun,'num',True)
        elif key == 'dataG':
            i = selection_cut.run_num(i,'G')
            event_weight =  {'event_weight':'NONE'}
        # leptons,photons,event = selection_cut.muon_nTuple(i,key,"D")
        # mass_mu = selection_cut.MASS_Muon(leptons,photons,'mu')
        # mass_gmumu = selection_cut.MASS_Muon(leptons,photons,'gmumu')
        # event_muon_variables.append(selection_cut.VARIABLES('muon',key,leptons,photons,event,mass_mu,mass_gmumu,event_weight))   
#ABCD
        leptons,photons,event = selection_cut.muon_nTuple(i,key,"A")
        event_muon_variables.append(selection_cut.VARIABLES('muon',key,leptons,photons,event,False,False,event_weight))   


    event_muon_phodrs[key] = event_muon_variables
    print(key,'event_phodrs',event_muon_phodrs[key][0])



#dataC 28min
#dataD 12min 
#zg0_22 #20min
#zg1_22 2min
#zg2_22 1min
#zg3_22 1min
#tt_22 25min
#ww,wz,zz 5min
#total 94min

zg3_22 event_phodrs {'photon_r9': <Array [0.969, 0.969, 0.955, ... 0.971, 0.98] type='319 * ?float32[parameters={"...'>, 'photon_seedGain': <Array [6, 12, 1, 1, 12, ... 12, 12, 12, 6, 12] type='319 * ?uint8[parameters={"...'>, 'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='319 * ?uint32[parameters={...'>}


In [5]:
# muon!!!!
# sample_names = ["dataC_muon","dataD_muon","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]
# sample_names = ["zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#,
# sample_names = ["tt_22","zz_22", "ww_22", "wz_22"]#"tt",
sample_names = ["zg3_22"]

# 初始化一个字典，用于存储合并后的数据
event_muon_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_muon_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_muon_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_muon_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_muon_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)

#dataE 1s
#zg1,zg2,zg3 15min
#tt 70min
#zz,ww,wz 20min

zg3_22


In [6]:
event_muon_final

{'zg3_22': {'photon_r9': <Array [0.969, 0.969, 0.955, ... 0.971, 0.975] type='14825 * ?float32[parameters...'>,
  'photon_seedGain': <Array [6, 12, 1, 1, 12, 1, ... 12, 6, 6, 6, 6] type='14825 * ?uint8[parameters=...'>,
  'run': <Array [1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1] type='14825 * ?uint32[parameters...'>}}

In [ ]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'mu_mass','gmumu_mass']#'photon_endcap','photon_barrel','npvsGood','Rho_Calo','Rho_tracker',,'dr_lg','charge1','charge2'
    data_subkeys = ['photon_r9','photon_seedGain','run']#['npvsGood','Rho_Calo','Rho_tracker']
    # other_subkeys = data_subkeys + ['nPU']#['generator_weight', 'event_weight']#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_muon1_pt','gen_muon2_pt','gen_muon1_eta','gen_muon2_eta','gen_gmumu','gen_mumu','LHEPdfWeight','LHEScaleWeight',,'gen_gmumu'
    other_subkeys = data_subkeys

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [11]:
# 存储数据
save_event_dict(event_muon_final, 'selection_muon_file/v2/A/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/B/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/C/event_muon_final')
# save_event_dict(event_muon_final, 'selection_muon_file/v2/D/event_muon_final')

# save_event_dict(event_muon_final, 'ABCD/ABCD_file/muon_channel/A/event_muon_final')


# Test

In [20]:
print(EVENT_ELE['zg0_22'][0].Photon.pixelSeed)

[[False], [], [], [True, False], ... False], [], [False], [True, True, False]]


In [22]:
print(EVENT_ELE['zg0_22'][0].Photon.pt)

[[12.8], [], [], [29.4, 21.3], [15.4], ... [12.4], [], [13.8], [37.7, 27.1, 10.7]]


In [6]:
###ELECTRON

import selection_cut
reload(selection_cut)
# sample = EVENT_ELE['dataD_ele'][10]
# Tag = 'dataD'
# sample = EVENT_ELE['wz_22'][0]
# Tag = 'wz'
sample = EVENT_ELE['zg3_22'][0]
Tag = 'zg3_22'
ABCD = "A"
# print('Start (Electron channel, sample:',Tag, ', region:',ABCD,') ',len(sample))
# ele0 = selection_cut.electron_nTuple(sample,Tag,ABCD)[0]
# print('HLT:',len(ele0),'Efficiency:',f"{len(ele0)/len(sample) * 100}%")
# ele1 = selection_cut.electron_nTuple(sample,Tag,ABCD)[1]
# ele1_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[2]
# print('pt, eta, medium id, charge, two ele:',len(ele1),'Efficiency:',f"{len(ele1)/len(ele0) * 100}%")
# ele2 = selection_cut.electron_nTuple(sample,Tag,ABCD)[3]
# ele2_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[4]
# print('Z mass window:',len(ele2),'Efficiency:',f"{len(ele2)/len(ele1) * 100}%")
# pho1 = selection_cut.electron_nTuple(sample,Tag,ABCD)[5]
# pho1_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[6]
# pho1_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[7]
# print('Photon pt,eta,id, at least one photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(ele2) * 100}%")
# pho2 = selection_cut.electron_nTuple(sample,Tag,ABCD)[8]
# pho2_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[9]
# pho2_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[10]
# print('Photon lepton dR, at least one photon:',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
# pho3 = selection_cut.electron_nTuple(sample,Tag,ABCD)[11]
# pho3_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[12]
# pho3_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[13]
# print('Photon prompt (pixelSeed), at least one photon:',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
# pho4 = selection_cut.electron_nTuple(sample,Tag,ABCD)[14]
# pho4_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[15]
# pho4_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[16]
# print('Photon prompt (Flav), at least one photon:',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
# pho5 = selection_cut.electron_nTuple(sample,Tag,ABCD)[17]
# pho5_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[18]
# pho5_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[19]
# print('Photon prompt (dR), only one photon:',len(pho5),'Efficiency:',f"{len(pho5)/len(pho4) * 100}%")

pho5_ele = selection_cut.electron_nTuple(sample,Tag,ABCD)[0]
pho5_pho = selection_cut.electron_nTuple(sample,Tag,ABCD)[1]
print('End',len(pho5_ele))


End 252


In [7]:
### MUON
import selection_cut
reload(selection_cut)
# sample = EVENT_MUON['dataD_muon'][10]
# Tag = 'dataD'
# sample = EVENT_MUON['wz_22'][0]
# Tag = 'wz'
sample = EVENT_MUON['zg3_22'][0]
Tag = 'zg3_22'
ABCD = 'A'
# print('Start (Muon channel, sample:',Tag, ', region:',ABCD,') ',len(sample))
# mu0 = selection_cut.muon_nTuple(sample,Tag,ABCD)[0]
# print('HLT:',len(mu0),'Efficiency:',f"{len(mu0)/len(sample) * 100}%")
# mu1 = selection_cut.muon_nTuple(sample,Tag,ABCD)[1]
# mu1_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[2]
# print('pt, eta, tight id&iso, charge, two mu:',len(mu1),'Efficiency:',f"{len(mu1)/len(mu0) * 100}%")
# mu2 = selection_cut.muon_nTuple(sample,Tag,ABCD)[3]
# mu2_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[4]
# print('Z mass window:',len(mu2),'Efficiency:',f"{len(mu2)/len(mu1) * 100}%")
# pho1 = selection_cut.muon_nTuple(sample,Tag,ABCD)[5]
# pho1_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[6]
# pho1_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[7]
# print('Photon pt,eta,i, at least one photon:',len(pho1),'Efficiency:',f"{len(pho1)/len(mu2) * 100}%")
# pho2 = selection_cut.muon_nTuple(sample,Tag,ABCD)[8]
# pho2_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[9]
# pho2_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[10]
# print('Photon lepton dR, at least one photon:',len(pho2),'Efficiency:',f"{len(pho2)/len(pho1) * 100}%")
# pho3 = selection_cut.muon_nTuple(sample,Tag,ABCD)[11]
# pho3_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[12]
# pho3_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[13]
# print('Photon prompt (pixelSeed), at least one photon:',len(pho3),'Efficiency:',f"{len(pho3)/len(pho2) * 100}%")
# pho4 = selection_cut.muon_nTuple(sample,Tag,ABCD)[14]
# pho4_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[15]
# pho4_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[16]
# print('Photon prompt (Flav), at least one photon:',len(pho4),'Efficiency:',f"{len(pho4)/len(pho3) * 100}%")
# pho5 = selection_cut.muon_nTuple(sample,Tag,ABCD)[17]
# pho5_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[18]
# pho5_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[19]
# print('Photon prompt (dR), at least one photon:',len(pho5),'Efficiency:',f"{len(pho5)/len(pho4) * 100}%")

pho5_mu = selection_cut.muon_nTuple(sample,Tag,ABCD)[0]
pho5_pho = selection_cut.muon_nTuple(sample,Tag,ABCD)[1]
print('End',len(pho5_mu))


End 319


In [5]:
sample = EVENT_ELE['zg3_22'][0]
Tag = 'zg3_22'
# sample = EVENT_ELE['dy_22'][10]
# Tag = 'dy_22'
ABCD = "A"

ele0 = sample[sample.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL]#
eta_cut = ele0.Electron[(abs(ele0.Electron.eta) < 2.5) & ~((abs(ele0.Electron.eta) > 1.4442) & (abs(ele0.Electron.eta) < 1.566 ))]
pt_cut = eta_cut[eta_cut.pt >= 20]
id_cut = pt_cut[pt_cut.cutBased >= 3] #0:fail, 1:veto, 2:loose, 3:medium, 4:tight
charge_num_cut = id_cut[(ak.sum(id_cut.pdgId,axis=1)==0) & (ak.num(id_cut.pdgId,axis=1)==2)]
ele1_ele =  charge_num_cut[ak.max(charge_num_cut.pt, axis=1) >= 25]
charge_num_event = ele0[(ak.sum(id_cut.pdgId,axis=1)==0) & (ak.num(id_cut.pdgId,axis=1)==2)]
ele1 = charge_num_event[ak.max(charge_num_cut.pt, axis=1) >= 25]
tem_ele = ele1_ele
ele_vectors = ak.zip(
{
"pt": tem_ele["pt"],
"eta": tem_ele["eta"],
"phi": tem_ele["phi"],
"mass": tem_ele["mass"],
},
with_name="PtEtaPhiMLorentzVector",
)
mass_ee = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass
ele2_ele = ele1_ele[(mass_ee >= 71) & (mass_ee <= 111)]
ele2 = ele1[(mass_ee >= 71) & (mass_ee <= 111)]
#photon
pt_cut = ele2.Photon[(ele2.Photon.pt > 30)]#&(ak.num(ele2.Photon.pt)<=2)
eta_cut = pt_cut[(abs(pt_cut.eta) < 2.5) & ~((abs(pt_cut.eta) > 1.4442) & (abs(pt_cut.eta) < 1.566))]
pt_id = (eta_cut.vidNestedWPBitmap[:] & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 1) & 1) 
scEta_id = (eta_cut.vidNestedWPBitmap[:] >> 2 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 3) & 1) 
HoE_id = (eta_cut.vidNestedWPBitmap[:] >> 4 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 5) & 1) 
siesie_id = (eta_cut.vidNestedWPBitmap[:] >> 6 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 7) & 1) 
chiso_id = (eta_cut.vidNestedWPBitmap[:] >> 8 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 9) & 1) 
neuiso_id = (eta_cut.vidNestedWPBitmap[:] >> 10 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 11) & 1) 
phoiso_id = (eta_cut.vidNestedWPBitmap[:] >> 12 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 13) & 1)
if ABCD == "A":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "B":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "C":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "D":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
# all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
id_cut = eta_cut[all_id_3]
photon_per_event = (ak.num(id_cut,axis=1) >= 1)
pho1 = ele2[photon_per_event]
pho1_ele = ele2_ele[photon_per_event]
pho1_pho = id_cut[photon_per_event]
lg_near, lg_dr = pho1_pho.nearest(pho1.Electron, axis=1, return_metric=True) #find the nearest electron near each photon
pho2 = pho1[ak.num(pho1_pho[lg_dr>0.5])>=1]
pho2_pho = pho1_pho[lg_dr>0.5]
pho2_pho = pho2_pho[ak.num(pho2_pho)>=1]
pho2_ele = pho1_ele[ak.num(pho1_pho[lg_dr>0.5])>=1]
pho3 = pho2[ak.num(pho2_pho[~pho2_pho.pixelSeed])>=1]
pho3_ele = pho2_ele[ak.num(pho2_pho[~pho2_pho.pixelSeed])>=1]
pho3_pho = pho2_pho[~pho2_pho.pixelSeed]
pho3_pho = pho3_pho[ak.num(pho3_pho)>=1]
if 'data' in Tag:
    print('pho3_ele',pho3_ele)
    print('pho3_pho',pho3_pho)
    print('pho3',pho3)
# return ele0,ele1,ele1_ele,ele2,ele2_ele,pho1,pho1_ele,pho1_pho,pho2,pho2_ele,pho2_pho,pho3,pho3_ele,pho3_pho#
if 'data' not in Tag:
    prompt_cut = (pho3_pho.genPartFlav == 1)
    pho4 = pho3[ak.num(pho3_pho[prompt_cut])>=1]
    pho4_ele = pho3_ele[ak.num(pho3_pho[prompt_cut])>=1]
    pho4_pho = pho3_pho[prompt_cut]
    pho4_pho = pho4_pho[ak.num(pho4_pho)>=1]
    gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
    photon5_near, photon5_gen_photon_dr = pho4_pho.nearest(gen_photon, axis=1, return_metric=True)
    mask = ak.fill_none(photon5_gen_photon_dr < 0.3, True)
    selected_photons5 = pho4_pho[mask]
    pho5 = pho4[ak.num(selected_photons5)>=1]
    pho5_ele = pho4_ele[ak.num(selected_photons5)>=1]
    pho5_pho = selected_photons5[ak.num(selected_photons5)>=1]
    print('pho5_ele',len(pho5_ele))
    print('pho5_pho',len(pho5_pho)) 
    print('pho5',len(pho5))
# return ele0,ele1,ele1_ele,ele2,ele2_ele,pho1,pho1_ele,pho1_pho,pho2,pho2_ele,pho2_pho,pho3,pho3_ele,pho3_pho,pho4,pho4_ele,pho4_pho,pho5,pho5_ele,pho5_pho#


pho5_ele 252
pho5_pho 252
pho5 252


In [15]:
#------------ Electron -----------
sample = EVENT_ELE['zg3_22'][0]
Tag = 'zg3_22'
# sample = EVENT_ELE['dy_22'][10]
# Tag = 'dy_22'
# ABCD = "D"
ABCD = "A"

#定义要用到的粒子
Event = sample
Electron = sample.Electron
Photon = sample.Photon

#罗列selection
#electron
ele_eta = (abs(Electron.eta) < 2.5) & ~((abs(Electron.eta) > 1.4442) & (abs(Electron.eta) < 1.566 ))
ele_pt = (Electron.pt >= 20)
ele_cutbased_id = (Electron.cutBased >= 3)

#photon
pho_pt = (Photon.pt >= 30)
pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
lg_near, lg_dr = Photon.nearest(Electron, axis=1, return_metric=True) #find the nearest electron near each photon
pho_drlg = lg_dr>0.5
pho_pixel = ~Photon.pixelSeed
pho_genpartFlav = (Photon.genPartFlav == 1)
# gen_photon = Event.GenPart[(Event.GenPart.pdgId==22) & Event.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
# genrecopho_near, genrecopho_dr = Photon.nearest(gen_photon, axis=1, return_metric=True)
# pho_drrecogen = ak.fill_none(genrecopho_dr < 0.3, True)


#联立选择条件
ele_selection = ele_eta & ele_pt & ele_cutbased_id 
photon_selection = pho_pt & pho_eta & pho_drlg 
if 'data' in Tag:
    photon_selection = photon_selection  
elif 'data' not in Tag and 'dy' not in Tag:
    photon_selection = photon_selection & pho_pixel & pho_genpartFlav 
elif 'dy' in Tag:
    photon_selection = photon_selection & (~pho_pixel | ~pho_genpartFlav)
else:
    print('Wrong sample!')


#处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25
good_electron = Electron[ele_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]
good_event = Event[(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]
good_photon = Photon[photon_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_selection])>=1)]


#最终处理事例，调用函数，mass在质量窗内，且通过trigger
#mass质量窗
tem_ele = good_electron
ele_vectors = ak.zip(
{
"pt": tem_ele["pt"],
"eta": tem_ele["eta"],
"phi": tem_ele["phi"],
"mass": tem_ele["mass"],
},
with_name="PtEtaPhiMLorentzVector",
)
mass_ee = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass

#photon id
pt_id = (good_photon.vidNestedWPBitmap[:] & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 1) & 1) 
scEta_id = (good_photon.vidNestedWPBitmap[:] >> 2 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 3) & 1) 
HoE_id = (good_photon.vidNestedWPBitmap[:] >> 4 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 5) & 1) 
siesie_id = (good_photon.vidNestedWPBitmap[:] >> 6 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 7) & 1) 
chiso_id = (good_photon.vidNestedWPBitmap[:] >> 8 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 9) & 1) 
neuiso_id = (good_photon.vidNestedWPBitmap[:] >> 10 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 11) & 1) 
phoiso_id = (good_photon.vidNestedWPBitmap[:] >> 12 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 13) & 1)
if ABCD == "A":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "B":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "C":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "D":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
else:
    print('Wrong ABCD value!')

selected_good_event = good_event[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
selected_good_electron = good_electron[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
selected_good_photon = good_photon[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]


print(len(Event))
print(len(selected_good_photon))


4698
252


In [ ]:
sample = EVENT_ELE['zg3_22'][0]
Tag = 'zg3_22'
# sample = EVENT_ELE['dy_22'][10]
# Tag = 'dy_22'
ABCD = "A"
# ABCD = "D"

#定义要用到的粒子
Event = sample
Photon = sample.Photon
Muon = sample.Muon

#罗列selection
#muon
muon_eta = (abs(Muon.eta) < 2.5) & ~((abs(Muon.eta) > 1.4442) & (abs(Muon.eta) < 1.566 ))
muon_pt = (Muon.pt >= 20)
muon_cutbased_id = (Muon.cutBased >= 3)

#photon
pho_pt = (Photon.pt >= 30)
pho_eta = (abs(Photon.eta) < 2.5) & ~((abs(Photon.eta) > 1.4442) & (abs(Photon.eta) < 1.566))
eg_near, eg_dr = Photon.nearest(Electron, axis=1, return_metric=True) #find the nearest electron near each photon
pho_dreg = eg_dr>0.5
mug_near, mug_dr = Photon.nearest(Muon, axis=1, return_metric=True) #find the nearest electron near each photon
pho_drmug = mug_dr>0.5
pho_pixel = ~Photon.pixelSeed
pho_pixel_dy = Photon.pixelSeed
pho_genpartFlav = (Photon.genPartFlav == 1)
pho_genpartFlav_dy = (Photon.genPartFlav != 1)
gen_photon = Event.GenPart[(Event.GenPart.pdgId==22) & Event.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
genrecopho_near, genrecopho_dr = Photon.nearest(gen_photon, axis=1, return_metric=True)
pho_drrecogen = ak.fill_none(genrecopho_dr < 0.3, True)
pho_drrecogen_dy = ak.fill_none(genrecopho_dr >= 0.3, True)

#联立选择条件
ele_selection = ele_eta & ele_pt & ele_cutbased_id 
photon_selection = pho_pt & pho_eta
if 'data' in Tag:
    photon_selection = photon_selection & pho_pixel 
elif 'data' not in Tag and 'dy' not in Tag:
    photon_selection = photon_selection & pho_pixel & pho_genpartFlav & pho_drrecogen
elif 'dy' in Tag:
    photon_selection = photon_selection & (pho_pixel_dy | pho_genpartFlav_dy | pho_drrecogen_dy)
else:
    print('Wrong sample!')
photon_muonchannel_selection = photon_selection & pho_dreg


#处理通过object挑选的事件 要求有两个electron，两个electron异号，最大的electron pt>=25

#最终处理事例，调用函数，mass在质量窗内，且通过trigger
#mass质量窗
tem_ele = good_electron
ele_vectors = ak.zip(
{
"pt": tem_ele["pt"],
"eta": tem_ele["eta"],
"phi": tem_ele["phi"],
"mass": tem_ele["mass"],
},
with_name="PtEtaPhiMLorentzVector",
)
mass_ee = (ele_vectors[:, 0] + ele_vectors[:, 1]).mass

#photon id
pt_id = (good_photon.vidNestedWPBitmap[:] & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 1) & 1) 
scEta_id = (good_photon.vidNestedWPBitmap[:] >> 2 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 3) & 1) 
HoE_id = (good_photon.vidNestedWPBitmap[:] >> 4 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 5) & 1) 
siesie_id = (good_photon.vidNestedWPBitmap[:] >> 6 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 7) & 1) 
chiso_id = (good_photon.vidNestedWPBitmap[:] >> 8 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 9) & 1) 
neuiso_id = (good_photon.vidNestedWPBitmap[:] >> 10 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 11) & 1) 
phoiso_id = (good_photon.vidNestedWPBitmap[:] >> 12 & 1) + 2*((good_photon.vidNestedWPBitmap[:] >> 13) & 1)
if ABCD == "A":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "B":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "C":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
elif ABCD == "D":
    all_id = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
else:
    print('Wrong ABCD value!')

good_electron = Electron[ele_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_muonchannel_selection])>=1)]
good_event = Event[(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_muonchannel_selection])>=1)]
good_photon = Photon[photon_muonchannel_selection][(ak.num(Electron[ele_selection])==2) & (ak.sum(Electron[ele_selection].pdgId,axis=1)==0) & (ak.fill_none(ak.max(Electron[ele_selection].pt, axis=1) >= 25, False)) & (ak.num(Photon[photon_muonchannel_selection])>=1)]

selected_good_event = good_event[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
selected_good_electron = good_electron[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]
selected_good_photon = good_photon[(good_event.HLT.Ele23_Ele12_CaloIdL_TrackIdL_IsoVL) & ((mass_ee >= 71) & (mass_ee <= 111)) & (ak.num(good_photon[all_id])>=1)]


print(len(Event))
print(len(selected_good_photon))


In [ ]:
mu0 = sample[sample.HLT.Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8]#
eta_cut = mu0.Muon[abs(mu0.Muon.eta) < 2.4]
pt_cut = eta_cut[eta_cut.pt >= 20]#[ ak.max(eta_cut.pt, axis=1) >= 25]
id_cut = pt_cut[(pt_cut.tightId) & (pt_cut.pfIsoId >= 4)]
charge_num_cut = id_cut[(ak.sum(id_cut.pdgId,axis=1)==0) & (ak.num(id_cut.pdgId,axis=1)==2)]
mu1_mu =  charge_num_cut[ak.max(charge_num_cut.pt, axis=1) >= 25]
charge_num_event = mu0[(ak.sum(id_cut.pdgId,axis=1)==0) & (ak.num(id_cut.pdgId,axis=1)==2)]
mu1 = charge_num_event[ak.max(charge_num_cut.pt, axis=1) >= 25]
tem_Muon = mu1_mu
muon_vectors = ak.zip(
{
    "pt": tem_Muon["pt"],
    "eta": tem_Muon["eta"],
    "phi": tem_Muon["phi"],
    "mass": tem_Muon["mass"],
},
with_name="PtEtaPhiMLorentzVector",
)
mass_mumu = (muon_vectors[:, 0] + muon_vectors[:, 1]).mass
mu2 = mu1[(mass_mumu >= 71) & (mass_mumu <= 111)]
mu2_mu = mu1_mu[(mass_mumu >= 71) & (mass_mumu <= 111)]
#photon
pt_cut = mu2.Photon[(mu2.Photon.pt > 30)]#&(ak.num(mu2.Photon.pt)<=2)
eta_cut = pt_cut[(abs(pt_cut.eta) < 2.5) & ~((abs(pt_cut.eta) > 1.4442) & (abs(pt_cut.eta) < 1.566))]
pt_id = (eta_cut.vidNestedWPBitmap[:] & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 1) & 1) 
scEta_id = (eta_cut.vidNestedWPBitmap[:] >> 2 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 3) & 1) 
HoE_id = (eta_cut.vidNestedWPBitmap[:] >> 4 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 5) & 1) 
siesie_id = (eta_cut.vidNestedWPBitmap[:] >> 6 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 7) & 1) 
chiso_id = (eta_cut.vidNestedWPBitmap[:] >> 8 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 9) & 1) 
neuiso_id = (eta_cut.vidNestedWPBitmap[:] >> 10 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 11) & 1) 
phoiso_id = (eta_cut.vidNestedWPBitmap[:] >> 12 & 1) + 2*((eta_cut.vidNestedWPBitmap[:] >> 13) & 1)
##tight
if ABCD == "A":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "B":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id == 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "C":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id == 3) & (neuiso_id == 3) & (phoiso_id == 3)
if ABCD == "D":
    all_id_3 = (pt_id == 3) & (scEta_id == 3) & (HoE_id == 3) & (siesie_id < 3) & (chiso_id < 3) & (neuiso_id == 3) & (phoiso_id == 3)
id_cut = eta_cut[all_id_3]
photon_per_event = (ak.num(id_cut,axis=1) >= 1)
pho1 = mu2[photon_per_event]
pho1_mu = mu2_mu[photon_per_event]
pho1_pho = id_cut[photon_per_event]
lg_near, lg_dr = pho1_pho.nearest(pho1.Muon, axis=1, return_metric=True)
pho2 = pho1[ak.num(pho1_pho[lg_dr>0.5])>=1]
pho2_mu = pho1_mu[ak.num(pho1_pho[lg_dr>0.5])>=1]
pho2_pho = pho1_pho[lg_dr>0.5]
pho2_pho = pho2_pho[ak.num(pho2_pho)>=1]
pho3 = pho2[ak.num(pho2_pho[~pho2_pho.pixelSeed])>=1]
pho3_mu = pho2_mu[ak.num(pho2_pho[~pho2_pho.pixelSeed])>=1]
pho3_pho = pho2_pho[~pho2_pho.pixelSeed]
pho3_pho = pho3_pho[ak.num(pho3_pho)>=1]
if 'data' in Tag:
    return pho3_mu,pho3_pho,pho3
    # return mu0,mu1,mu1_mu,mu2,mu2_mu,pho1,pho1_mu,pho1_pho,pho2,pho2_mu,pho2_pho,pho3,pho3_mu,pho3_pho
if 'data' not in Tag:
    prompt_cut = (pho3_pho.genPartFlav == 1)
    pho4 = pho3[ak.num(pho3_pho[prompt_cut])>=1]
    pho4_mu = pho3_mu[ak.num(pho3_pho[prompt_cut])>=1]
    pho4_pho = pho3_pho[prompt_cut]
    pho4_pho = pho4_pho[ak.num(pho4_pho)>=1]
    gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
    photon5_near, photon5_gen_photon_dr = pho4_pho.nearest(gen_photon, axis=1, return_metric=True)
    mask = ak.fill_none(photon5_gen_photon_dr < 0.3, True)
    selected_photons5 = pho4_pho[mask]
    pho5 = pho4[ak.num(selected_photons5)>=1]
    pho5_mu = pho4_mu[ak.num(selected_photons5)>=1]
    pho5_pho = selected_photons5[ak.num(selected_photons5)>=1]
    return pho5_mu,pho5_pho,pho5
    # return mu0,mu1,mu1_mu,mu2,mu2_mu,pho1,pho1_mu,pho1_pho,pho2,pho2_mu,pho2_pho,pho3,pho3_mu,pho3_pho,pho4,pho4_mu,pho4_pho,pho5,pho5_mu,pho5_pho

    

In [17]:
import selection_cut
reload(selection_cut)
event_phodrs = {key: {} for key in EVENT}


for key in EVENT.keys():
    event_variables = []
    # fiducial_variables = []
    for i in EVENT[key]:
#muon_cut
        if 'data' not in key:
            event_weight = {'event_weight':np.sign(i.Generator.weight)}
            mu0 = selection_cut.muon(i,'num',True)
        elif key == 'dataF':
            murun = selection_cut.muon(i,'run_num','F')
            event_weight =  {'event_weight':'NONE'}
            mu0 = selection_cut.muon(murun,'num',True)
        elif key == 'dataG':
            murun = selection_cut.muon(i,'run_num','G')
            event_weight =  {'event_weight':'NONE'}
            mu0 = selection_cut.muon(murun,'num',True)            
        mu1 = selection_cut.muon(mu0,'trigger','double')
        mu2,event2 = selection_cut.muon(mu1,'cutBasedID','tight')
        mu3 = selection_cut.muon(mu2,'pf','tight')
        mu4 = selection_cut.muon(mu3,'eta','2.4')
        mu5 = selection_cut.muon(mu4,'pt',event2)
        mu6 = selection_cut.muon(mu5,'pid',True)
        mass_mumu = selection_cut.MASS(mu6,'mu')
        mu_mass = selection_cut.muon(mu6,'mass',mass_mumu)
#photon_cut
        pho1 = selection_cut.photon(mu_mass, 'pt', '30')
        pho2 = selection_cut.photon(pho1, 'eta', '2.5')
        pho3 = selection_cut.photon(pho2, 'id', 'tight') #tight #B #C #D
        if 'data' in key:
            # event_variables.append(selection_cut.VARIABLES(pho3,mass_mumu,mass_gmumu,key,event_weight))
            lg_near, lg_dr = pho3.Photon.nearest(pho3.Muon, axis=1, return_metric=True)
            fsr = selection_cut.muon(pho3,'fsr',lg_dr)
            mass_mumu = selection_cut.MASS(fsr,'mu')
            mass_gmumu = selection_cut.MASS(fsr,'gmumu')
            event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,False, False,key,event_weight))
        else:
            lg_near, lg_dr = pho3.Photon.nearest(pho3.Muon, axis=1, return_metric=True)
            fsr = selection_cut.muon(pho3,'fsr',lg_dr)
            # pho4 = selection_cut.photon(pho3,'prompt',True) #看细节
            # pho4 = selection_cut.photon(pho3,'prompt',False) #看细节
            # gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
            # photon4_near, photon4_gen_photon_dr = gen_photon.nearest(pho4.Photon, axis=1, return_metric=True)
            # mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
            # pho5 = pho4[ak.num(pho4.Photon[ak.any(mask == True,axis=1)]) == 1]
            # lg_near, lg_dr = pho5.Photon.nearest(pho5.Muon, axis=1, return_metric=True)
            # fsr = selection_cut.muon(pho5,'fsr',lg_dr)
            mass_mumu = selection_cut.MASS(fsr,'mu')
            mass_gmumu = selection_cut.MASS(fsr,'gmumu')
            event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,key,event_weight))
            # gen_gmumu = selection_cut.gen_mass(fsr,'gmumu')
            # gen_mumu = selection_cut.gen_mass(fsr,'mu')
            # event_variables.append(selection_cut.VARIABLES(fsr,mass_mumu,mass_gmumu,gen_gmumu,gen_mumu,key,event_weight))
    event_phodrs[key] = event_variables
    print(key,'event_phodrs',event_phodrs[key][0])



#dataF 74m  
#dataG 10m
#zg0 30m
#mc 98m

zg0 event_phodrs {'photon_pt': <Array [[42], [36], ... [32.6], [33.8]] type='188 * option[var * float32[paramet...'>, 'photon_endcap': <Array [[False], [False], ... [True], [False]] type='188 * option[var * bool[par...'>, 'photon_barrel': <Array [[True], [True], ... [False], [True]] type='188 * option[var * bool[param...'>, 'Photon_genPartFlav': <Array [[1], [1], [1], ... [1], [1], [1]] type='188 * option[var * uint8[paramet...'>, 'Gen_statusflag': <Array [[10625, 22913, 4481, ... 12352, 12352]] type='188 * option[var * int32[p...'>, 'generator_weight': <Array [1, 1, 1, -1, 1, 1, ... -1, 1, 1, 1, 1] type='188 * ?float32'>, 'event_weight': <Array [1, 1, 1, -1, 1, 1, ... 1, -1, 1, 1, 1] type='202914 * float32'>}
zg1 event_phodrs {'photon_pt': <Array [[95.3], [53.3], ... [86.5], [56.1]] type='521 * option[var * float32[par...'>, 'photon_endcap': <Array [[True], [False], ... [True], [True]] type='521 * option[var * bool[param...'>, 'photon_barrel': <Array [[False], [True], ... [False], [Fa

In [18]:
#清理内存
# import gc

# del EVENT
# gc.collect()

In [19]:
# 样本名称列表
# sample_names = ["dataF", "dataG","zg1", "zg2", "zg3", "tt", "zz", "ww", "wz"]
# sample_names = ["zg0"]
sample_names = ["zg0","zg1", "zg2", "zg3"]
# sample_names = ["tt", "zz", "ww", "wz"]#"tt",
# sample_names = ["dataF", "dataG"]
# sample_names = ["dy1","dy2"]#,"dy""zg2", "zg3"

# 初始化一个字典，用于存储合并后的数据
event_final = {}

# 遍历每个样本名称
for sample_name in sample_names:
    print(sample_name)
    # 初始化一个字典，用于存储当前样本的合并数据
    event_final[sample_name] = {}

    # 获取第一个条目中的信息键
    info_keys = list(event_phodrs[sample_name][0].keys())

    # 遍历每个信息键
    for info_key in info_keys:
        # 从 event_muon_phodrs 中提取当前样本和信息键的数据
        data_to_concatenate = [entry[info_key] for entry in event_phodrs[sample_name]]

        # 使用 ak.concatenate 将数据连接在一起
        event_final[sample_name][info_key] = ak.concatenate(data_to_concatenate)

#zg0 10min
#zg1 
#dy1 dy2 277min

zg0
zg1
zg2
zg3


In [20]:
def save_event_dict(event_dict, file_prefix):
    # data_subkeys = ['photon_pt', 'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'muon_mass','gmumu_mass','photon_endcap','photon_barrel','npvsGood','Rho_Calo','Rho_tracker']#,'npvsGood','Rho_Calo','Rho_tracker',,'dr_lg','charge1','charge2'
    data_subkeys = ['Photon_genPartFlav','Gen_statusflag','photon_pt','photon_endcap','photon_barrel',]
    other_subkeys = data_subkeys + ['generator_weight', 'event_weight',]#'LHEPdfWeight','LHEScaleWeight','PSWeight','GenDressedLepton','GenIsolatedPhoton','gen_gmumu','gen_mumu' 'gen_muon1_pt','gen_muon2_pt','gen_muon1_eta','gen_muon2_eta','gen_gmumu','gen_mumu','LHEPdfWeight','LHEScaleWeight',,'gen_gmumu'
    # other_subkeys = data_subkeys

    for key, value in event_dict.items():
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        for subkey in subkeys:
            ak.to_parquet(value[subkey], f'{file_prefix}_{key}_{subkey}.parquet')



In [21]:
# 存储数据
# save_event_dict(event_final, 'selection_file/event_final')
save_event_dict(event_final, 'ABCD_file/closure_test/A/event_final')
# save_event_dict(event_final, 'ABCD_file/B/event_final')
# save_event_dict(event_final, 'test_file/D/event_final')

# TEST

In [12]:
sample = EVENT['zg3'][0]
pass_mu = (sample['GenDressedLepton'].pt > 10) 
pass_pho = (sample['GenIsolatedPhoton'].pt > 10)
sample_tem_test = sample['GenDressedLepton'][abs(sample['GenDressedLepton'].eta) < 2.4]#2.4
in_fiducial_1 = sample[(ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2)]
out_fiducial_1 = sample[~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))]
in_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)==2]
out_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)!=2]


out_fiducial_all = sample[(~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))) | (ak.num(sample_tem_test.eta)!=2)]

In [13]:
print(len(in_fiducial_1))
print(len(out_fiducial_1))
print(len(out_fiducial_all))

292
347
356


In [10]:
sample = EVENT['zg3'][0]
pass_mu = (sample['GenDressedLepton'].pt > 10) 
pass_pho = (sample['GenIsolatedPhoton'].pt > 10)
in_fiducial_1 = sample[(ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2)]
out_fiducial_1 = sample[~((ak.num(pass_pho,axis=1)==1) & (ak.num(pass_mu,axis=1)==2))]
sample_tem = in_fiducial_1['GenDressedLepton'][abs(in_fiducial_1['GenDressedLepton'].eta) < 2.4]#2.4
in_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)==2]
out_fiducial_2 = in_fiducial_1[ak.num(sample_tem.eta)!=2]

In [6]:
print(len(in_fiducial_1))
print(len(out_fiducial_1))
print(len(in_fiducial_2))
print(len(out_fiducial_2))

292
347
283
9


In [6]:
import selection_cut
reload(selection_cut)

i = EVENT['zg3'][0]
event_weight = {'event_weight':np.sign(i.Generator.weight)}
mu0 = selection_cut.muon(i,'num',True)          
mu1 = selection_cut.muon(mu0,'trigger','double')
mu2,event2 = selection_cut.muon(mu1,'cutBasedID','tight')
mu3 = selection_cut.muon(mu2,'pf','tight')
mu4 = selection_cut.muon(mu3,'eta','2.4')
mu5 = selection_cut.muon(mu4,'pt',event2)
mu6 = selection_cut.muon(mu5,'pid',True)
mass_mumu = selection_cut.MASS(mu6,'mu')
mu_mass = selection_cut.muon(mu6,'mass',mass_mumu)
#photon_cut
pho1 = selection_cut.photon(mu_mass, 'pt', '30')
pho2 = selection_cut.photon(pho1, 'eta', '2.5')
pho3 = selection_cut.photon(pho2, 'id', 'tight') #tight
pho4 = selection_cut.photon(pho3,'prompt',True)
gen_photon = pho4.GenPart[(pho4.GenPart.pdgId==22) & pho4.GenPart.hasFlags(['isLastCopy',"isPrompt"])]
photon4_near, photon4_gen_photon_dr = gen_photon.nearest(pho4.Photon, axis=1, return_metric=True)
mask = ak.fill_none(photon4_gen_photon_dr < 0.3, True)
pho5 = pho4[ak.num(pho4.Photon[ak.any(mask == True,axis=1)]) == 1]
# event_variables.append(selection_cut.VARIABLES(pho5,mass_mumu,mass_gmumu,key,event_weight))
lg_near, lg_dr = pho5.Photon.nearest(pho5.Muon, axis=1, return_metric=True)
fsr = selection_cut.muon(pho5,'fsr',lg_dr)
gen_mumu = selection_cut.gen_mass(fsr,'mu')
gen_mumu


#dataF 74m  
#dataG 10m
#zg0 30m
#mc 98m

<Array [83.8, 91.6, 89.6, ... 90.3, 90.1, 92.4] type='30 * ?float32'>

In [58]:
gen_mumu = selection_cut.gen_mass(fsr,'mu')

In [57]:
ak.max(samples['GenDressedLepton']['pdgId'])

13

In [53]:
gen_muon_vectors_0 = ak.zip(
    {
        "pt": samples['GenDressedLepton']['pt'],
        "eta": samples['GenDressedLepton']['eta'],
        "phi": samples['GenDressedLepton']['phi'],
        "mass": samples['GenDressedLepton']['mass']
    },
    with_name="PtEtaPhiMLorentzVector"
)
# GEN_gamma = GEN[GEN['pdgId'] == 22]
gen_photon_vector_0 = ak.zip(
    {
        "pt": samples['GenIsolatedPhoton']['pt'],
        "eta": samples['GenIsolatedPhoton']['eta'],
        "phi": samples['GenIsolatedPhoton']['phi'],
        "mass": samples['GenIsolatedPhoton']['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2]
gen_photon_vector = gen_photon_vector_0[ak.num(gen_muon_vectors_0)==2]
# gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2][ak.num(gen_photon_vector_0['pt'])==1]
# gen_photon_vector = gen_photon_vector_0[ak.num(gen_muon_vectors_0)==2][ak.num(gen_photon_vector_0['pt'])==1]

In [65]:
samples = fsr
gen_muon_vectors_0 = ak.zip(
        {
            "pt": samples['GenDressedLepton']['pt'],
            "eta": samples['GenDressedLepton']['eta'],
            "phi": samples['GenDressedLepton']['phi'],
            "mass": samples['GenDressedLepton']['mass']
        },
        with_name="PtEtaPhiMLorentzVector"
    )
gen_muon_vectors = gen_muon_vectors_0[ak.num(gen_muon_vectors_0)==2]
# GEN_gamma = GEN[GEN['pdgId'] == 22]
gen_photon_vector = ak.zip(
    {
        "pt": fsr['GenIsolatedPhoton']['pt'],
        "eta": fsr['GenIsolatedPhoton']['eta'],
        "phi": fsr['GenIsolatedPhoton']['phi'],
        "mass": fsr['GenIsolatedPhoton']['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] ).mass
# invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] + gen_photon_vector[:,0]).mass
invariant_mass

<Array [85.2, 92.7, 91.4, ... 83.2, 89.6, 85.7] type='89 * ?float32'>

In [63]:
samples = fsr
gen_muon_idxs = samples['Muon']["genPartIdx"]
GEN = samples['GenPart']
gen_muon_vectors = ak.zip(
    {
        "pt": GEN[gen_muon_idxs]['pt'],
        "eta": GEN[gen_muon_idxs]['eta'],
        "phi": GEN[gen_muon_idxs]['phi'],
        "mass": GEN[gen_muon_idxs]['mass']
    },
    with_name="PtEtaPhiMLorentzVector"
)
# 获取gen level photon的四动量信息
gen_photon_idx = samples.Photon[:, 0]["genPartIdx"]
GEN = ak.flatten(samples['GenPart'])
gen_photon_vector = ak.zip(
    {
        "pt": GEN[gen_photon_idx]['pt'],
        "eta": GEN[gen_photon_idx]['eta'],
        "phi": GEN[gen_photon_idx]['phi'],
        "mass": GEN[gen_photon_idx]['mass'],
    },
    with_name="PtEtaPhiMLorentzVector"
)
invariant_mass = (gen_muon_vectors[:, 0] + gen_muon_vectors[:, 1] ).mass
invariant_mass

<Array [92.6, 90.9, 90.1, ... 93.5, 92.6, 90.4] type='6499 * ?float32'>

In [66]:
        GEN = samples['GenPart']
        GEN_muons = GEN[abs(GEN['pdgId']) == 13]
        GEN_gamma = GEN[GEN['pdgId'] == 22]
        GEN_VARIABLES = {
            'gen_muon1_pt':GEN_muons['pt'][:,0],
            'gen_muon2_pt':GEN_muons['pt'][:,1],
            'gen_muon1_eta':GEN_muons['eta'][:,0],
            'gen_muon2_eta':GEN_muons['eta'][:,1],
            'gen_gamma_pt':GEN_gamma['pt'][:,0],
            'gen_gamma_eta':GEN_gamma['eta'][:,0],
            }

/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: overflow encountered in sinh
  result = getattr(ufunc, method)(
/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in multiply
  result = getattr(ufunc, method)(
/home/pku/muty/anaconda3/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: overflow encountered in cosh
  result = getattr(ufunc, method)(


<Array [671, nan, nan, 400, ... nan, nan, 815] type='6499 * ?float32'>